In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from surprise import SVD, KNNBasic, Reader, Dataset, accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold, GridSearchCV
from surprise.prediction_algorithms.baseline_only import BaselineOnly


In [4]:
df = pd.read_csv('bigframe.csv')

#### Remove all but the most recent one User/Beer combo for multiple such checkins

In [6]:
df.drop_duplicates(subset=['user_id', 'beer_id'], inplace=True)  # keep_first (default) means keep most recent checkin

In [135]:
df.shape  # remaining from 1.42M prev

(1296064, 28)

In [8]:
reader = Reader(rating_scale=(0.25, 5.0))

### Put this into surprise's format (user, beer, rating)

In [10]:
# The columns must correspond to user_id, beer_id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'beer_id', 'rating_user']], reader)

In [12]:
# define a cross-validation iterator
kf = KFold(n_splits=5)
algo = SVD()


In [165]:

for trainset, testset in kf.split(data):
    # Train the algorithm on the trainset, and predict ratings for the testset
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Then compute RMSE
    accuracy.rmse(predictions)

RMSE: 0.4726
RMSE: 0.4729
RMSE: 0.4733
RMSE: 0.4758
RMSE: 0.4729


In [17]:
predictions[:11]

[Prediction(uid=171496, iid=3034980, r_ui=4.25, est=4.4778698277391804, details={'was_impossible': False}),
 Prediction(uid=1803223, iid=1828774, r_ui=3.25, est=3.5844314206255463, details={'was_impossible': False}),
 Prediction(uid=267320, iid=399609, r_ui=4.0, est=3.981760322915016, details={'was_impossible': False}),
 Prediction(uid=2924460, iid=1187911, r_ui=4.0, est=3.6026735719768013, details={'was_impossible': False}),
 Prediction(uid=1749282, iid=2686615, r_ui=3.5, est=3.6181331817534472, details={'was_impossible': False}),
 Prediction(uid=3292172, iid=3970, r_ui=3.25, est=3.9172385217788266, details={'was_impossible': False}),
 Prediction(uid=4396115, iid=4057, r_ui=4.0, est=3.2826608617101911, details={'was_impossible': False}),
 Prediction(uid=959498, iid=1473393, r_ui=3.5, est=3.6003136089757715, details={'was_impossible': False}),
 Prediction(uid=267320, iid=9681, r_ui=4.0, est=4.3961044757993522, details={'was_impossible': False}),
 Prediction(uid=1279351, iid=647032, r_u

How does that compare to just guessing the global mean rating for every beer?  
First we have to remove the beers that don't have global ratings.  We could  
theoretically use the mean of all user ratings in place of the missing global  
ratings, but for beers with few checkins, that would cheat  
by skewing toward the rating of the user being predicted.

In [21]:
global_rated = df[['rating_user', 'rating_global']]

In [22]:
global_rated.shape

(1296064, 2)

In [98]:
global_rated.max()

rating_user      5.00000
rating_global    4.90672
dtype: float64

In [23]:
global_rated.min()

rating_user      0.25
rating_global    0.00
dtype: float64

Whoops, went to the trouble of setting NaN global ratings to 0.0

In [24]:
global_rated = global_rated[global_rated.rating_global > 0]

In [25]:
global_rated.shape

(1089604, 2)

In [26]:
global_rated.min()

rating_user      0.25000
rating_global    1.00423
dtype: float64

In [29]:
# calculate the rmse using global rating as prediction
gr = global_rated
diffs = gr.rating_global.values - gr.rating_user.values
sum_errs_sq = np.dot(diffs, diffs)
rmse = np.sqrt(sum_errs_sq / len(diffs))
rmse

0.51243794703341272

So at least the Surprise rmse was lower than that of (the `global_rated` subset of) the whole population.
  
Quick check of what pct of ratings had globals: 

In [109]:
print(f'{round(len(global_rated) * 100 / len(df), 3)} pct of checkins had global ratings.')

84.07 pct of checkins had global ratings.


In [131]:
# might as well keep track of evaluation results
methods = ['naively predict global_mean_rating']
rmses = [0.512]
results = pd.DataFrame({'method': methods,
                        'rmse': rmses})

def add_results_row(meth_name, rmse_result):
    methods.append(meth_name)
    rmses.append(rmse_result)
    results = pd.DataFrame({'method': methods,
                            'rmse': rmses})
    return results

# make sure function works
results = add_results_row('Surprise SVD with default params', 0.474)
results.tail()

,method,rmse
0,naively predict global_mean_rating,0.512
1,Surprise SVD with default params,0.474


In [170]:
# append result rows here and check tail to make sure all good
results = add_results_row('SVD lr=.02 epochs=20 reg=0.1', 0.466)
results.tail()

,method,rmse
0,naively predict global_mean_rating,0.512
1,Surprise SVD with default params,0.474
2,naive mean plus userbias,0.456
3,naive mean plus smart userbias,0.434
4,SVD lr=.02 epochs=20 reg=0.1,0.466


Let's just make sure the error for the testset alone was about the same (.512) as for whole population.

In [112]:
# make a dictionary to map all beer_id's to global ratings, where available
bidrates = df[df.rating_global > 0].groupby('beer_id').rating_global.mean()
bidrates.head()

beer_id
1     3.27892
2     3.49213
8     3.81242
10    3.38719
14    3.77430
Name: rating_global, dtype: float64

In [114]:
dbr = dict(bidrates)  # map beer_id's to glob_ratings

In [123]:
# Now line up all the beers in just the test set,
#   filter out the ones with no global, and put them
#   alongside all their ratings.

globrates = pd.Series([dbr.get(triple[1], 0) for triple in testset])
userrates = pd.Series([triple[2] for triple in testset])
userrates = userrates[globrates > 0]
globrates = globrates[globrates > 0]
# calc rmse
diffs = userrates.values - globrates.values
sum_errs_sq = np.dot(diffs, diffs)
rmse = np.sqrt(sum_errs_sq / len(diffs))
rmse

0.51115640157523945

Let's factor in the user's rating generosity, which should help the naive guess a bit.

In [125]:
# use groupby to get each rater's mean, 
# then use that mapping to add a new column to the end of the df for each checkin
userbias = dict(df.groupby('user_id').rating_user.mean())
df['userbias'] = df.user_id.map(lambda u: userbias[u]) - df.rating_user.mean()

In [129]:
bias_df = df[['rating_user', 'rating_global', 'userbias']]
bias_df = bias_df[bias_df.rating_global > 0]
# add the user's generosity vs the mean
diffs = bias_df.rating_global.values + bias_df.userbias.values - bias_df.rating_user.values
sum_errs_sq = np.dot(diffs, diffs)
rmse = np.sqrt(sum_errs_sq / len(diffs))
rmse

0.45625249039480498

That helped a lot.  A single 'feature' is already better than the Surprise SVD model.
But the calculated userbias is simply how a user rates vis-a-vis the average rating,
and doesn't take into account the fact that certain raters rate higher-rated beers.
If user A lives where beers globally rated 4.5 stars are available, and gives them 4.25 stars,
while user B rates average beers 4.0 where others rated them 3.75, why does user A get
an extra quarter star added to his predictions compared to user B?  I.e. our generosity measure
should maybe account for which beers are being rated.

In [141]:
# for each user, find the difference between their ratings for their beers vs. the world's ratings for same ones
userbeers = dict(userrates.rating_user.mean() - userrates.rating_global.mean())

In [146]:
df['userbeerbias'] = df[df.rating_global > 0].user_id.map(lambda u: userbeers[u])

In [152]:
bias_df = df[['rating_user', 'rating_global', 'userbeerbias']]
bias_df = bias_df[bias_df.rating_global > 0]
# add the user's generosity vs the mean
diffs = bias_df.rating_global.values + bias_df.userbeerbias.values - bias_df.rating_user.values
sum_errs_sq = np.dot(diffs, diffs)
rmse = np.sqrt(sum_errs_sq / len(diffs))
rmse 

0.43369005337892269

That does better.  Let's see if it aligns with Surprise's BaselineOnly model.

In [155]:
baseline = BaselineOnly()

In [156]:
baseline.fit(trainset)

Estimating biases using als...


In [157]:
predictions = baseline.test(testset)
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.4688


0.46881078934132836

hmmmmm, thought that should be 0.434....guess surprise is using estimations where i used actual global ratings.

If Surprise can't keep up with simple baseline predictions, at least while using default parameters, maybe it's time for some GridSearching.

In [ ]:
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.001, 0.002, 0.005, 0.01],
              'reg_all': [0.2, 0.5, 1.0, 2.0]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

In [164]:
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.470300063255
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}


Not much improvement there from the default params (.474 improved to .470), but since the best  
params out of the grid were all at the end of the param range (15 epochs was the most in the grid,  
0.01 was the highest learning rate, and 0.2 was the lowest regularization), let's try fitting  
with the next step out in the directions of those params.


In [166]:
algo = SVD(lr_all=0.02, n_epochs=20, reg_all=0.1, verbose=True)
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

In [167]:
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.4664


0.46640651047237058

Better than 0.470, so keep trying, with higher step rate, and lower regularization, until things reverse.

In [169]:
algo = SVD(lr_all=0.03, n_epochs=20, reg_all=0.05, verbose=True)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.4683


0.46830288741631865

That didn't take long.  So store the best params from previous fit into the results table.

In [173]:
sim_options = {'name': 'pearson'}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
preds = algo.test(testset)
accuracy.rmse(preds)

Computing the pearson similarity matrix...


/Applications/anaconda/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning: invalid value encountered in sqrt
  sim = construction_func[name](*args)


Done computing similarity matrix.
RMSE: 0.5886


0.58863602712876439

How do individual user tastes determine which features correlate with ratings?

In [174]:
df.columns

Index(['checkin_id', 'beer_id', 'user_id', 'rating_user', 'brewery_name',
       'beer_name', 'beer_style', 'brewery_id', 'brewery_type',
       'brewery_country', 'brewery_city', 'brewery_state', 'brewery_lat',
       'brewery_lon', 'venue_lat', 'venue_lon', 'venue_city', 'venue_country',
       'venue_state', 'venue_cat', 'venue_id', 'checkin_comment', 'venue_type',
       'rating_global', 'beer_description', 'abv', 'date', 'userbias',
       'userbeerbias'],
      dtype='object')

In [175]:
df.shape

(1296064, 29)

===========================================================================================================